<a href="https://colab.research.google.com/github/profteachkids/CHE2064/blob/master/Demo_AdiabaticFlash.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://raw.githubusercontent.com/profteachkids/CHE2064/master/vsc.py
!wget https://raw.githubusercontent.com/profteachkids/CHE2064/master/trust_ncg.py
!wget https://raw.githubusercontent.com/profteachkids/CHE2064/master/che_tools.py
!pip install DotMap

--2020-09-09 00:22:43--  https://raw.githubusercontent.com/profteachkids/CHE2064/master/vsc.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3549 (3.5K) [text/plain]
Saving to: ‘vsc.py’

vsc.py              100%[===================>]   3.47K  --.-KB/s    in 0s      

2020-09-09 00:22:44 (53.4 MB/s) - ‘vsc.py’ saved [3549/3549]

--2020-09-09 00:22:44--  https://raw.githubusercontent.com/profteachkids/CHE2064/master/trust_ncg.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2871 (2.8K) [text/plain]
Saving to: ‘trust_ncg.py’

trust_ncg.py 

In [2]:
from dotmap import DotMap
import jax.numpy as jnp
import jax
import vsc
import trust_ncg as tncg
import che_tools as che

In [3]:
p = che.Props(['Ethanol', 'Isopropanol', 'Water'])

In [4]:
def model(c):
    c=DotMap(c)
    x=che.qtox(c.Vq)
    y=che.qtox(c.Lq)
    z=che.qtox(c.Fq)
    V = x * c.Vtot
    L = y * c.Ltot
    F = z * c.Ftot
    mass_balance = (F - V - L)
    mass_balance = (mass_balance/jnp.sum(F))**2

    FH = p.Hmix( (1.-c.q)*F, c.q*F, c.FT)
    VH = p.Hmix(V, 0, c.flashT)
    LH = p.Hmix(0, L, c.flashT)
    energy_balance = (FH - VH - LH)
    energy_balance = (energy_balance/jnp.sum(FH))**2

    fugacity = (x*p.Pvap(c.flashT) - y*c.flashP)
    fugacity = (fugacity/jnp.sum(y*c.flashP))**2

    return jnp.sum(mass_balance + energy_balance + fugacity)

In [5]:
s=DotMap()
s.Ftot=10
s.Fq = che.xtoq([1/3, 1/3, 1/3])
s.FT = 350
s.flashP= 101325
s.q = 0.5 #fraction liquid

v=DotMap()
v.Vq = che.xtoq([0.4, 0.35, 0.25])
v.Lq = che.xtoq([0.25, 0.35, 0.4])
v.flashT = 300
v.Vtot = s.Ftot/2
v.Ltot = s.Ftot/2

[Traced<ShapedArray(float64[], weak_type=True):JaxprTrace(level=-1/1)>, Traced<ShapedArray(float64[], weak_type=True):JaxprTrace(level=-1/1)>, Traced<ShapedArray(float64[], weak_type=True):JaxprTrace(level=-1/1)>]


/usr/local/lib/python3.6/dist-packages/jax/lib/xla_bridge.py:130: UserWarning: No GPU/TPU found, falling back to CPU.
  warnings.warn('No GPU/TPU found, falling back to CPU.')


In [6]:
c=vsc.VSC(v,s)
modelf = jax.jit(c.transform(model))
x_min, f = tncg.minimize(modelf,c.x, verbosity=1)

0:4, f: 0.3182956617493208
1:3, f: 0.2951542974058736
2:5, f: 0.2895570764862791
3:4, f: 0.2805715753193567
4:5, f: 0.25219124942674875
5:4, f: 0.1934232645111691
6:2, f: 0.1934232645111691
7:2, f: 0.1934232645111691
8:4, f: 0.17067381117819186
9:4, f: 0.15238815883140183
10:4, f: 0.13187458640068903
11:5, f: 0.08792602805737397
12:6, f: 0.012659903933506063
13:6, f: 0.0005386452972950885
14:6, f: 3.525986138217572e-06
15:6, f: 4.6408293175236376e-10
Final results:
f: 8.83358843701275e-18
x: [ 4.01011698e-01  3.36410535e-01  5.28612421e+00 -4.14090787e-01
 -3.21380334e-01  4.71387582e+00  3.59966484e+02]


In [7]:
v_sol = c.xtoc(x_min)
df=vsc.todf(v_sol)
df


[350.] FT
[0. 0.] Fq
[10.] Ftot
[0.4010117  0.33641054] Lq
[5.28612421] Ltot
[-0.41409079 -0.32138033] Vq
[4.71387582] Vtot
[101325.] flashP
[359.96648358] flashT
[0.5] q


,,"(vector2, 1)","(vector2, 2)"
FT,350,,
Fq,,0,0
Ftot,10,,
Lq,,0.401012,0.336411
Ltot,5.28612,,
Vq,,-0.414091,-0.32138
Vtot,4.71388,,
flashP,101325,,
flashT,359.966,,
q,0.5,,


In [10]:
 print(f'Vapor composition: {che.qtox(v_sol.Vq)}')
 print(f'Liquid composition: {che.qtox(v_sol.Lq)}')
 

Vapor composition: [0.27699768 0.30390635 0.41909597]
Liquid composition: [0.38357038 0.3595747  0.25685492]


,,"(vector2, 1)","(vector2, 2)"
FT,350,,
Fq,,0,0
Ftot,10,,
Lq,,0.401012,0.336411
Ltot,5.28612,,
Vq,,-0.414091,-0.32138
Vtot,4.71388,,
flashP,101325,,
flashT,359.966,,
q,0.5,,
